In [1]:
data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "schoffelen2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

In [2]:
# from utils.compression import compress_directories, decompress_directories

# for base_path, batch_type in data_partition.items():
#     decompress_directories(
#         source_path=f'downloaded_data/{base_path}',
#         destination_path=f'data/{base_path}',
#         checksum_file_name="checksums.txt",
#         delete_compressed_files=True,
#         num_workers=26
#     )

In [3]:
from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=False,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

In [4]:
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=100,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=20,
    baseline_window=0.5,
    notch_filter=True,
    scaling="standard",
    delay=0.30,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/ablation/delay/0_30",
    clear_cache=False,
    cache_name="cache",
)

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
SimpleConv initialized with 8448160 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 7973376


In [5]:
try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2) // 2,
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

2024-12-21 01:07:02,600	INFO worker.py:1821 -- Started a local Ray instance.
Training Epoch 1: 100%|██████████| 135/135 [04:19<00:00,  1.92s/it]


Testing at epoch 1
Test unseen_subject completed. Accuracy: 0.0905, Top 1: 0.1405, Top 5: 0.3490, Top 10: 0.4750, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0730, Top 1: 0.1062, Top 5: 0.3470, Top 10: 0.4983, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0488, Top 1: 0.0975, Top 5: 0.3100, Top 10: 0.4588, Perplexity: 0.0000
Epoch 1 completed in 4.80m. 0.04m per recording.


Training Epoch 2: 100%|██████████| 135/135 [05:11<00:00,  2.31s/it]


Testing at epoch 2
Test unseen_subject completed. Accuracy: 0.1175, Top 1: 0.1760, Top 5: 0.4030, Top 10: 0.5295, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1289, Top 1: 0.1805, Top 5: 0.4448, Top 10: 0.6061, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1088, Top 1: 0.1750, Top 5: 0.3962, Top 10: 0.5175, Perplexity: 0.0000
Epoch 2 completed in 5.65m. 0.04m per recording.


Training Epoch 3: 100%|██████████| 135/135 [05:05<00:00,  2.26s/it]


...

Testing at epoch 24
Test unseen_subject completed. Accuracy: 0.1600, Top 1: 0.2350, Top 5: 0.4660, Top 10: 0.5730, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1712, Top 1: 0.2283, Top 5: 0.5087, Top 10: 0.6476, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1288, Top 1: 0.1975, Top 5: 0.4300, Top 10: 0.5475, Perplexity: 0.0000
Epoch 24 completed in 6.02m. 0.04m per recording.


Training Epoch 25: 100%|██████████| 135/135 [05:43<00:00,  2.54s/it]


Testing at epoch 25
Test unseen_subject completed. Accuracy: 0.1330, Top 1: 0.1965, Top 5: 0.4300, Top 10: 0.5450, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1310, Top 1: 0.1774, Top 5: 0.4550, Top 10: 0.5974, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0975, Top 1: 0.1550, Top 5: 0.3563, Top 10: 0.4925, Perplexity: 0.0000
Epoch 25 completed in 6.11m. 0.05m per recording.


Training Epoch 26: 100%|██████████| 135/135 [05:39<00:00,  2.51s/it]


Testing at epoch 26
Test unseen_subject completed. Accuracy: 0.1505, Top 1: 0.2225, Top 5: 0.4680, Top 10: 0.5725, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1512, Top 1: 0.2106, Top 5: 0.4831, Top 10: 0.6250, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1162, Top 1: 0.1787, Top 5: 0.4000, Top 10: 0.5288, Perplexity: 0.0000
Epoch 26 completed in 6.05m. 0.04m per recording.
Early stopping at epoch 26. Highest top 10 accuracy at epoch 15.
Training completed.
unseen_subject: Acc: 0.2190, Top 1: 0.3040, Top 5: 0.5525, Top 10: 0.6365
unseen_task: Acc: 0.2385, Top 1: 0.3105, Top 5: 0.6042, Top 10: 0.7376
unseen_both: Acc: 0.1763, Top 1: 0.2625, Top 5: 0.4862, Top 10: 0.5863


## Baseline corr 10s

In [6]:
del session.logger
del session

from pickle import NONE
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "schoffelen2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=False,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=100,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=20,
    baseline_window=10,
    notch_filter=True,
    scaling="standard",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/ablation/baseline_10_sec",
    clear_cache=True,
    cache_name="cache",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2) // 2,
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

Loading Gwilliams2023 with batch type audio
Cleared cache for study gwilliams2023
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
SimpleConv initialized with 8448160 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 7973376


Training Epoch 1: 100%|██████████| 135/135 [34:40<00:00, 15.41s/it] 


Testing at epoch 1
Test unseen_subject completed. Accuracy: 0.0815, Top 1: 0.1250, Top 5: 0.3520, Top 10: 0.4880, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0753, Top 1: 0.1097, Top 5: 0.3556, Top 10: 0.5264, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0688, Top 1: 0.1275, Top 5: 0.3287, Top 10: 0.4625, Perplexity: 0.0000
Epoch 1 completed in 41.04m. 0.30m per recording.


Training Epoch 2: 100%|██████████| 135/135 [04:42<00:00,  2.09s/it]


Testing at epoch 2
Test unseen_subject completed. Accuracy: 0.0470, Top 1: 0.0835, Top 5: 0.2705, Top 10: 0.3995, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0582, Top 1: 0.0891, Top 5: 0.2746, Top 10: 0.4205, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0462, Top 1: 0.0837, Top 5: 0.2913, Top 10: 0.4075, Perplexity: 0.0000
Epoch 2 completed in 5.14m. 0.04m per recording.


...

Testing at epoch 18
Test unseen_subject completed. Accuracy: 0.1555, Top 1: 0.2405, Top 5: 0.4945, Top 10: 0.6055, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.2291, Top 1: 0.3078, Top 5: 0.6113, Top 10: 0.7523, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1588, Top 1: 0.2412, Top 5: 0.4775, Top 10: 0.5975, Perplexity: 0.0000
Epoch 18 completed in 5.80m. 0.04m per recording.


Training Epoch 19: 100%|██████████| 135/135 [05:22<00:00,  2.39s/it]


Testing at epoch 19
Test unseen_subject completed. Accuracy: 0.1335, Top 1: 0.2095, Top 5: 0.4620, Top 10: 0.5745, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1771, Top 1: 0.2467, Top 5: 0.5441, Top 10: 0.6935, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1200, Top 1: 0.1963, Top 5: 0.4637, Top 10: 0.5737, Perplexity: 0.0000
Epoch 19 completed in 5.82m. 0.04m per recording.


Training Epoch 20: 100%|██████████| 135/135 [05:24<00:00,  2.41s/it]


Testing at epoch 20
Test unseen_subject completed. Accuracy: 0.1640, Top 1: 0.2505, Top 5: 0.4945, Top 10: 0.6030, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1628, Top 1: 0.2238, Top 5: 0.5078, Top 10: 0.6530, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1250, Top 1: 0.1963, Top 5: 0.4350, Top 10: 0.5537, Perplexity: 0.0000
Epoch 20 completed in 5.85m. 0.04m per recording.
Early stopping at epoch 20. Highest top 10 accuracy at epoch 9.
Training completed.
unseen_subject: Acc: 0.1830, Top 1: 0.2715, Top 5: 0.5120, Top 10: 0.6175
unseen_task: Acc: 0.1971, Top 1: 0.2678, Top 5: 0.5807, Top 10: 0.7298
unseen_both: Acc: 0.1862, Top 1: 0.2575, Top 5: 0.5150, Top 10: 0.6325
